In [1]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [2]:
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [4]:
test_df = pd.read_csv('emnist-byclass-test.csv/emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

In [5]:
train_df = pd.read_csv('emnist-byclass-train.csv/emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

In [6]:
train_y = train_df.pop('Target')
train_x = train_df
test_y = test_df.pop('Target')
test_x = test_df
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = train_x.values.reshape([-1, 28, 28, 1])
test_x = test_x.values.reshape([-1, 28, 28, 1])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [7]:
train_x[0].shape

(28, 28, 1)

In [8]:
lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])

In [9]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [10]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

Epoch 1/100
17449/17449 [==============================] - 62s 3ms/step - loss: 0.5873 - accuracy: 0.8067 - val_loss: 0.4771 - val_accuracy: 0.8335
Epoch 2/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4542 - accuracy: 0.8389 - val_loss: 0.4494 - val_accuracy: 0.8415
Epoch 3/100
17449/17449 [==============================] - 52s 3ms/step - loss: 0.4351 - accuracy: 0.8443 - val_loss: 0.4507 - val_accuracy: 0.8360
Epoch 4/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4254 - accuracy: 0.8471 - val_loss: 0.4377 - val_accuracy: 0.8438
Epoch 5/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4173 - accuracy: 0.8494 - val_loss: 0.4326 - val_accuracy: 0.8444
Epoch 6/100
17449/17449 [==============================] - 71s 4ms/step - loss: 0.4134 - accuracy: 0.8501 - val_loss: 0.4380 - val_accuracy: 0.8437
Epoch 7/100
17449/17449 [==============================] - 104s 6ms/step - loss: 0.4097 - accuracy: 0.8511 - val

In [11]:
lenet_5_model.evaluate(test_x, test_y)

3636/3636 [==============================] - 6s 2ms/step - loss: 0.4179 - accuracy: 0.8510


[0.4179256856441498, 0.8509998321533203]